In [ ]:
# Read in an image
import numpy as np
from PIL import Image

# Youll need to put an image here. I've got a picture of apples
img = Image.open("../../assets/test_imgs/apples.png").convert("L")
img.load()
img = np.asarray(img)

In [ ]:
# First lets try a left to right join, so we'll make two arrays representing our overlap
overlap_width = img.shape[0]
overlap_height = 10

img1 = img[:overlap_width, 10:10 + overlap_height]
img2 = img[:overlap_width, 20:20 + overlap_height]

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(2, 6))
axes[0].imshow(img1, cmap='gray')
axes[1].imshow(img2, cmap='gray')

for axis in axes:
    axis.axis("off")


In [ ]:
# Dynamic programming problem - we want to find a seam from the top to the bottom of the image
# that minimizes the cost of the seam, where the cost is the difference between the two images

# Fixed Dynamic programming problem
cost = np.sqrt((img1 - img2) ** 2)

H, W = cost.shape
dp = np.full_like(cost, np.inf, dtype=float)

# The first row of the dp array is just the cost of the first row
dp[0] = cost[0]

for i in range(1, H):
    for j in range(W):
        # Current cost at position (i, j)
        current_cost = cost[i, j]
        
        # Check three possible previous positions that can reach (i, j)
        if j > 0:  # Can come from (i-1, j-1) - diagonal left
            dp[i, j] = min(dp[i, j], dp[i-1, j-1] + current_cost)
        
        # Can come from (i-1, j) - directly above
        dp[i, j] = min(dp[i, j], dp[i-1, j] + current_cost)
        
        if j < W - 1:  # Can come from (i-1, j+1) - diagonal right
            dp[i, j] = min(dp[i, j], dp[i-1, j+1] + current_cost)

In [ ]:
seam = np.zeros(H, dtype=int)
seam[-1] = np.argmin(dp[-1])
for i in reversed(range(H - 1)):
    j = seam[i + 1]
    options = [j]
    if j > 0: options.append(j - 1)
    if j < W - 1: options.append(j + 1)
    seam[i] = min(options, key=lambda x: dp[i, x])

In [ ]:
# Create stitched image using the seam
stitched = np.zeros_like(img1)

for i in range(H):
    seam_pos = seam[i]
    # Use img1 for pixels to the left of the seam, img2 for pixels to the right
    stitched[i, :seam_pos] = img1[i, :seam_pos]
    stitched[i, seam_pos:] = img2[i, seam_pos:]

# Visualize the stitching result
fig, axes = plt.subplots(1, 3, figsize=(10, 8))

# Original images
axes[0].imshow(img1, cmap="gray")
axes[0].set_title("Image 1 (Left Overlap)")
axes[0].plot(seam, range(H), "red", linewidth=2)

axes[1].imshow(img2, cmap="gray")
axes[1].set_title("Image 2 (Right Overlap)")
axes[1].plot(seam, range(H), "red", linewidth=2)

# Stitched result
axes[2].imshow(stitched, cmap="gray")
axes[2].set_title("Stitched Result")



for axis in axes.flat:
    axis.axis("off")

plt.tight_layout()
plt.show()
